NHS Digital produce ["business rules"](https://digital.nhs.uk/data-and-information/data-collections-and-data-sets/data-collections/quality-and-outcomes-framework-qof/quality-and-outcome-framework-qof-business-rules/quality-and-outcomes-framework-qof-business-rules-v42-2019-2020-baseline-release) to support the correct identification and counts of patients to support General Practice reimbursement under the quality and outcomes framework. They produce a word document which in turn links to a "ref set" that can be interrogated on the NHS SnoMed browser.

Here we set out to interrograte the AF QoF [business rules refset for medicines](https://termbrowser.nhs.uk/?perspective=full&conceptId1=12463501000001109&edition=uk-edition&release=v20190601&server=https://termbrowser.nhs.uk/sct-browser-api/snomed&langRefset=999001261000000100,999000691000001104) against the NHS dm+d for discrepancies. The refset has an id number of 12463501000001109 and has "188 members"

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [2]:
##First we identify VMPs.
sql = '''SELECT
  *
FROM
  `ebmdatalab.dmd.vmp`
WHERE
  bnf_code LIKE "0208020Y0%" ##RIVAROXABAN
  OR bnf_code LIKE "0208020X0%" ##DABIGATRAN
  OR bnf_code LIKE "0208020Z0%" ##APIXABAN
  OR bnf_code LIKE "0208020AA%" ##EDOXABAN
  OR bnf_code LIKE "0208020V0%" ##WARFARIN
  OR bnf_code LIKE "0208020H0%" ##ACENOCOUMARLO
  OR bnf_code LIKE "0208020N0%" ###PHENIDIONE'''
    
vmp_anticoag = bq.cached_read(sql, csv_path='vmp_anticoagp.csv')
vmp_anticoag.head(10)

,id,vpiddt,vpidprev,vtm,invalid,nm,abbrevnm,basis,nmdt,nmprev,...,glu_f,pres_f,cfc_f,non_avail,non_availdt,df_ind,udfs,udfs_uom,unit_dose_uom,bnf_code
0,8797911000001107,NaN,NaN,48603004,False,Warfarin 10mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAVAV
1,8798011000001109,NaN,NaN,48603004,False,Warfarin 10mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAVAV
2,8798511000001101,NaN,NaN,48603004,False,Warfarin 3mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAMAM
3,8798611000001102,NaN,NaN,48603004,False,Warfarin 3mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAMAM
4,8798711000001106,NaN,NaN,48603004,False,Warfarin 5mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAIAI
5,8798811000001103,NaN,NaN,48603004,False,Warfarin 5mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAIAI
6,8798111000001105,NaN,NaN,48603004,False,Warfarin 1mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAANAN
7,8798211000001104,NaN,NaN,48603004,False,Warfarin 1mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAANAN
8,8798311000001107,NaN,NaN,48603004,False,Warfarin 2mg/5ml oral solution,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAJAJ
9,8798411000001100,NaN,NaN,48603004,False,Warfarin 2mg/5ml oral suspension,NaN,1,NaN,NaN,...,False,False,False,NaN,NaN,2,NaN,NaN,NaN,0208020V0AAAJAJ


In [3]:
vmp_anticoag.count()

id               43
vpiddt            2
vpidprev          2
vtm              43
invalid          43
nm               43
abbrevnm          0
basis            43
nmdt              1
nmprev            1
basis_prev        1
nmchange          1
combprod          1
pres_stat        43
sug_f            43
glu_f            43
pres_f           43
cfc_f            43
non_avail         1
non_availdt       1
df_ind           43
udfs             24
udfs_uom         24
unit_dose_uom    24
bnf_code         43
dtype: int64

In [4]:
vmp_anticoag.nunique(axis = 0)

id               43
vpiddt            1
vpidprev          2
vtm               7
invalid           1
nm               43
abbrevnm          0
basis             2
nmdt              1
nmprev            1
basis_prev        1
nmchange          1
combprod          1
pres_stat         1
sug_f             2
glu_f             1
pres_f            1
cfc_f             1
non_avail         1
non_availdt       1
df_ind            3
udfs              1
udfs_uom          2
unit_dose_uom     2
bnf_code         38
dtype: int64

The VMP dmd file give us 43 products of which none are invalid......

Now we mst look for AMPs

In [5]:
sql = '''SELECT
  *
FROM
  `ebmdatalab.dmd.amp`
WHERE
  bnf_code LIKE "0208020Y0%" ##RIVAROXABAN
  OR bnf_code LIKE "0208020X0%" ##DABIGATRAN
  OR bnf_code LIKE "0208020Z0%" ##APIXABAN
  OR bnf_code LIKE "0208020AA%" ##EDOXABAN
  OR bnf_code LIKE "0208020V0%" ##WARFARIN
  OR bnf_code LIKE "0208020H0%" ##ACENOCOUMARLO
  OR bnf_code LIKE "0208020N0%" ###PHENIDIONE
  '''
    
amp_anticoag = bq.cached_read(sql, csv_path='amp_anticoagp.csv')
amp_anticoag.head(10)

,id,invalid,vmp,nm,abbrevnm,descr,nmdt,nm_prev,supp,lic_auth,lic_auth_prev,lic_authchange,lic_authchangedt,combprod,flavour,ema,parallel_import,avail_restrict,bnf_code
0,29902111000001100,False,29903211000001100,Lixiana 15mg tablets,NaN,Lixiana 15mg tablets (Daiichi Sankyo UK Ltd),NaN,NaN,2081301000001107,1,NaN,NaN,NaN,NaN,NaN,True,False,1,0208020AABBAAAA
1,29902411000001105,False,29903311000001108,Lixiana 30mg tablets,NaN,Lixiana 30mg tablets (Daiichi Sankyo UK Ltd),NaN,NaN,2081301000001107,1,NaN,NaN,NaN,NaN,NaN,True,False,1,0208020AABBABAB
2,29902711000001104,False,29903411000001101,Lixiana 60mg tablets,NaN,Lixiana 60mg tablets (Daiichi Sankyo UK Ltd),NaN,NaN,2081301000001107,1,NaN,NaN,NaN,NaN,NaN,True,False,1,0208020AABBACAC
3,738511000001103,False,319740004,Sinthrome 1mg tablets,NaN,Sinthrome 1mg tablets (Merus Labs Luxco II S.a...,NaN,NaN,22480511000001101,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020H0BBAAAA
4,3781211000001108,False,319745009,Phenindione 10mg tablets,NaN,Phenindione 10mg tablets (Advanz Pharma),NaN,NaN,24129411000001106,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020N0AAAAAA
5,3781411000001107,False,319745009,Phenindione 10mg tablets,NaN,Phenindione 10mg tablets (Alliance Healthcare ...,NaN,NaN,2089901000001107,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020N0AAAAAA
6,3782311000001109,False,319746005,Phenindione 25mg tablets,NaN,Phenindione 25mg tablets (Advanz Pharma),NaN,NaN,24129411000001106,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020N0AAABAB
7,3782511000001103,False,319746005,Phenindione 25mg tablets,NaN,Phenindione 25mg tablets (Alliance Healthcare ...,NaN,NaN,2089901000001107,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020N0AAABAB
8,421411000001103,False,319733000,Warfarin 1mg tablets,NaN,Warfarin 1mg tablets (A A H Pharmaceuticals Ltd),NaN,NaN,3144701000001104,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020V0AAAAAA
9,303111000001107,False,319733000,Warfarin 1mg tablets,NaN,Warfarin 1mg tablets (Actavis UK Ltd),NaN,NaN,3875201000001104,1,NaN,NaN,NaN,NaN,NaN,False,False,1,0208020V0AAAAAA


In [6]:
amp_anticoag.count()

id                  128
invalid             128
vmp                 128
nm                  128
abbrevnm              0
descr               128
nmdt                  0
nm_prev               0
supp                128
lic_auth            128
lic_auth_prev         0
lic_authchange        0
lic_authchangedt      0
combprod              1
flavour               0
ema                 128
parallel_import     128
avail_restrict      128
bnf_code            128
dtype: int64

In [7]:
amp_anticoag.nunique(axis = 0)

id                  128
invalid               1
vmp                  43
nm                   47
abbrevnm              0
descr               128
nmdt                  0
nm_prev               0
supp                 31
lic_auth              2
lic_auth_prev         0
lic_authchange        0
lic_authchangedt      0
combprod              1
flavour               0
ema                   2
parallel_import       1
avail_restrict        4
bnf_code             42
dtype: int64

The AMP gives us 128 products which is 171 products in total. 17 short of the NHS Digtial ref set............we hve previously been told that Sno Med products are only authored after dm+d products are authored

Questions
- Are these correct dmd tables from BQ
- Are we missing products with sql rules?
- How do we download from SnoMed browser to compare the two to see where discrepancies lie.

In [8]:
import requests
import json

url = "https://termbrowser.nhs.uk/sct-browser-api/snomed/uk-edition/v20190601/concepts/12463501000001109/members?limit=300&paginate=1"

data = requests.get(url).json()
records = data['members']
ref_set = pd.DataFrame(records)
ref_set.head(5)

,_id,active,conceptId,defaultTerm,definitionStatus,isLeafInferred,isLeafStated,module
0,5d1e656d1592260353d3112a,True,10417011000001103,Warfarin 500microgram tablets (Arrow Generics ...,Primitive,True,True,999000011000001104
1,5d1e65751592260353d3254d,True,772211000001101,Warfarin 3mg tablets (Kent Pharmaceuticals Ltd...,Primitive,True,True,999000011000001104
2,5d1e65881592260353d35d1d,True,12959411000001100,Warfarin 4.16mg/5ml oral solution (Special Ord...,Primitive,True,True,999000011000001104
3,5d1e65a71592260353d3b0bc,True,10418411000001106,Warfarin 3mg tablets (Arrow Generics Ltd) (pro...,Primitive,True,True,999000011000001104
4,5d1e65b41592260353d3d4d2,True,584211000001100,Warfarin 3mg tablets (Unichem Plc) (product),Primitive,True,True,999000011000001104


In [9]:
ref_set.count()

_id                 188
active              188
conceptId           188
defaultTerm         188
definitionStatus    188
isLeafInferred      188
isLeafStated        188
module              188
dtype: int64

In [10]:
ref_set.nunique(axis = 0)

_id                 188
active                2
conceptId           188
defaultTerm         188
definitionStatus      2
isLeafInferred        2
isLeafStated          1
module                2
dtype: int64

Merge AMP and VMP

In [11]:
amp_vmp = vmp_anticoag.append(amp_anticoag)
amp_vmp.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,abbrevnm,avail_restrict,basis,basis_prev,bnf_code,cfc_f,combprod,descr,df_ind,ema,...,pres_stat,sug_f,supp,udfs,udfs_uom,unit_dose_uom,vmp,vpiddt,vpidprev,vtm
0,NaN,NaN,1.0,NaN,0208020V0AAAVAV,False,NaN,NaN,2.0,NaN,...,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48603004.0
1,NaN,NaN,1.0,NaN,0208020V0AAAVAV,False,NaN,NaN,2.0,NaN,...,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48603004.0
2,NaN,NaN,1.0,NaN,0208020V0AAAMAM,False,NaN,NaN,2.0,NaN,...,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48603004.0
3,NaN,NaN,1.0,NaN,0208020V0AAAMAM,False,NaN,NaN,2.0,NaN,...,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48603004.0
4,NaN,NaN,1.0,NaN,0208020V0AAAIAI,False,NaN,NaN,2.0,NaN,...,1.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48603004.0


In [12]:
amp_vmp.drop(["abbrevnm","basis", "basis_prev", "cfc_f", "combprod", 
              "descr", "df_ind", "ema", "flavour", "glu_f", "lic_auth",
              "lic_auth_prev", "lic_authchange", "lic_authchangedt", 
              "pres_stat", "sug_f", "supp", "udfs", "udfs_uom", "unit_dose_uom",
             "nm_prev", "nmchange", "nmdt", "nmprev", "non_availdt",
             "non_avail", "parallel_import", "pres_f"], axis=1, inplace=True)

In [13]:
amp_vmp.head(5)

,avail_restrict,bnf_code,id,invalid,nm,vmp,vpiddt,vpidprev,vtm
0,NaN,0208020V0AAAVAV,8797911000001107,False,Warfarin 10mg/5ml oral solution,NaN,NaN,NaN,48603004.0
1,NaN,0208020V0AAAVAV,8798011000001109,False,Warfarin 10mg/5ml oral suspension,NaN,NaN,NaN,48603004.0
2,NaN,0208020V0AAAMAM,8798511000001101,False,Warfarin 3mg/5ml oral solution,NaN,NaN,NaN,48603004.0
3,NaN,0208020V0AAAMAM,8798611000001102,False,Warfarin 3mg/5ml oral suspension,NaN,NaN,NaN,48603004.0
4,NaN,0208020V0AAAIAI,8798711000001106,False,Warfarin 5mg/5ml oral solution,NaN,NaN,NaN,48603004.0


In [14]:
ref_set.rename(columns={"conceptId":"id"}, inplace=True)

ref_set.head()

,_id,active,id,defaultTerm,definitionStatus,isLeafInferred,isLeafStated,module
0,5d1e656d1592260353d3112a,True,10417011000001103,Warfarin 500microgram tablets (Arrow Generics ...,Primitive,True,True,999000011000001104
1,5d1e65751592260353d3254d,True,772211000001101,Warfarin 3mg tablets (Kent Pharmaceuticals Ltd...,Primitive,True,True,999000011000001104
2,5d1e65881592260353d35d1d,True,12959411000001100,Warfarin 4.16mg/5ml oral solution (Special Ord...,Primitive,True,True,999000011000001104
3,5d1e65a71592260353d3b0bc,True,10418411000001106,Warfarin 3mg tablets (Arrow Generics Ltd) (pro...,Primitive,True,True,999000011000001104
4,5d1e65b41592260353d3d4d2,True,584211000001100,Warfarin 3mg tablets (Unichem Plc) (product),Primitive,True,True,999000011000001104


In [15]:
ref_set['id'] = ref_set.id.astype('int64')

In [16]:
ref_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 8 columns):
_id                 188 non-null object
active              188 non-null bool
id                  188 non-null int64
defaultTerm         188 non-null object
definitionStatus    188 non-null object
isLeafInferred      188 non-null bool
isLeafStated        188 non-null bool
module              188 non-null object
dtypes: bool(3), int64(1), object(4)
memory usage: 8.0+ KB


In [18]:
df_all = ref_set.merge(amp_vmp.drop_duplicates(), on=['id'], 
                   how='left', indicator=True)

In [19]:
df_all.head()

,_id,active,id,defaultTerm,definitionStatus,isLeafInferred,isLeafStated,module,avail_restrict,bnf_code,invalid,nm,vmp,vpiddt,vpidprev,vtm,_merge
0,5d1e656d1592260353d3112a,True,10417011000001103,Warfarin 500microgram tablets (Arrow Generics ...,Primitive,True,True,999000011000001104,9.0,0208020V0AAASAS,False,Warfarin 500microgram tablets,3.197360e+08,NaN,NaN,NaN,both
1,5d1e65751592260353d3254d,True,772211000001101,Warfarin 3mg tablets (Kent Pharmaceuticals Ltd...,Primitive,True,True,999000011000001104,9.0,0208020V0AAABAB,False,Warfarin 3mg tablets,3.197340e+08,NaN,NaN,NaN,both
2,5d1e65881592260353d35d1d,True,12959411000001100,Warfarin 4.16mg/5ml oral solution (Special Ord...,Primitive,True,True,999000011000001104,6.0,0208020V0AAAPAP,False,Warfarin 4.16mg/5ml oral solution,1.301701e+16,NaN,NaN,NaN,both
3,5d1e65a71592260353d3b0bc,True,10418411000001106,Warfarin 3mg tablets (Arrow Generics Ltd) (pro...,Primitive,True,True,999000011000001104,9.0,0208020V0AAABAB,False,Warfarin 3mg tablets,3.197340e+08,NaN,NaN,NaN,both
4,5d1e65b41592260353d3d4d2,True,584211000001100,Warfarin 3mg tablets (Unichem Plc) (product),Primitive,True,True,999000011000001104,1.0,0208020V0AAABAB,False,Warfarin 3mg tablets,3.197340e+08,NaN,NaN,NaN,both


In [20]:
diff_df = df_all.loc[(df_all['_merge'] == 'left_only')]
diff_df

,_id,active,id,defaultTerm,definitionStatus,isLeafInferred,isLeafStated,module,avail_restrict,bnf_code,invalid,nm,vmp,vpiddt,vpidprev,vtm,_merge
9,5d1e65d11592260353d41bef,True,29953511000001108,Acenocoumarol 1mg tablets (Lexon (UK) Ltd) (pr...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29,5d1e66181592260353d4d5ac,True,30270611000001104,Eliquis 2.5mg tablets (Waymade Healthcare Plc)...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
31,5d1e662a1592260353d503ae,True,17637911000001100,Sinthrome 1mg tablets (Waymade Healthcare Plc)...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
37,5d1e66701592260353d591c0,True,24669411000001100,Xarelto 15mg tablets (Waymade Healthcare Plc) ...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
40,5d1e669d1592260353d5fbaf,False,21749211000001108,Acenocoumarol 1mg tablets (Waymade Healthcare ...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
41,5d1e66a61592260353d612cb,True,36908411000001103,Xarelto 20mg tablets (Originalis B.V.) (product),Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
42,5d1e66b01592260353d62be2,True,36820511000001100,Pradaxa 75mg capsules (Originalis B.V.) (product),Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
73,5d1e67bb1592260353d8b28c,True,18555311000001109,Pradaxa 110mg capsules (Waymade Healthcare Plc...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
75,5d1e67c31592260353d8cb55,True,18067311000001105,Sinthrome 1mg tablets (Lexon (UK) Ltd) (product),Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
88,5d1e67ff1592260353d96ab4,True,23677711000001107,Pradaxa 150mg capsules (Waymade Healthcare Plc...,Primitive,True,True,999000011000001104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


All these are primitive and look like parallel imports - is there something in the BQ table that I have not used the right one ebmdatalab.dmd.amp